## Importando bibliotecas

In [22]:
import pandas as pd
import numpy as np

## Processando dataset socioeconômico

In [23]:
df = pd.read_csv('../data/raw/socioeconomico.csv', sep=';')
df['MICROREGIAO'] = df['MICROREGIAO'].astype('category')
df['BAIRRO'] = df['BAIRRO'].astype('category')
df['RPA'] = df['RPA'].astype('category')
print(df.dtypes)

df.to_hdf('../data/processed/socioeconomico.h5', key='data', format="table")

BAIRRO                   category
RPA                      category
MICROREGIAO              category
DIST_MARCO_ZERO           float64
HECTARES                  float64
HABITANTES                  int64
NUM_HOMENS                  int64
POR_HOMENS                float64
NUM_MULHERES                int64
POR_MULHERES              float64
0_4ANOS                     int64
0_4ANOS%                  float64
5_14ANOS                    int64
5_14ANOS%                 float64
15_17ANOS                   int64
15_17ANOS%                float64
18_24ANOS                   int64
18_24ANOS%                float64
25_59ANOS                   int64
25_59ANOS%                float64
60_ANOS                     int64
60_ANOS%                  float64
BRANCA                    float64
PRETA                     float64
PARDA                     float64
AMARELA                   float64
INDIGENA                  float64
ALFABETIZADOS             float64
CRESCIMENTO_ANUAL         float64
DENSIDADE     

## Processando dataset de calendário

In [24]:
df = pd.read_csv('../data/raw/calendario.csv', sep=';')
df = df.drop(['description', 'raw_description', 'link'], axis=1)
df.columns = ['DATA', 'NOME', 'TIPO', 'CODIGO_TIPO']
df['NOME'] = df['NOME'].apply(lambda x: x.upper()).astype('category')
df['TIPO'] = df['TIPO'].apply(lambda x: x.upper()).astype('category')
df['CODIGO_TIPO'] = df['CODIGO_TIPO'].astype('category')
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')
print(df.dtypes)

df.to_hdf('../data/processed/calendario.h5', key='data', format="table")

DATA           datetime64[ns]
NOME                 category
TIPO                 category
CODIGO_TIPO          category
dtype: object


## Processando dataset pluviométrico

In [25]:
df = pd.read_csv('../data/raw/pluviometrico.csv', sep=';')
df = df.drop(['Acumulado'], axis=1)
df['MES'] = df['Mês/Ano'].apply(lambda x: x.split('/')[0])
df['ANO'] = df['Mês/Ano'].apply(lambda x: x.split('/')[1])
df = df.drop(['Posto', 'Mês/Ano'], axis=1)
group = df.groupby(['ANO', 'MES']).mean().reset_index()
df = pd.melt(group, ['ANO', 'MES'], group.columns.values[2:], 'DIA', 'INDICE')

map_month = {
    'jan': '01',
    'fev': '02',
    'mar': '03',
    'abr': '04',
    'mai': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'set': '09',
    'out': '10',
    'nov': '11',
    'dez': '12'}

df['MES'] = df['MES'].apply(lambda x: map_month[x])
df['DATA'] = df[['DIA', 'MES', 'ANO']].apply(lambda x: x[0] + '/' + x[1] + '/' + x[2], axis=1)
df = df.drop(['DIA', 'MES', 'ANO'], axis=1)[['DATA', 'INDICE']]
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y', errors='coerce')
df = df.dropna()
print(df.dtypes)

df.to_hdf('../data/processed/pluviometrico.h5', key='data')

DATA      datetime64[ns]
INDICE           float64
dtype: object


## Processando dataset de localização

In [26]:
df = pd.read_csv("../data/raw/156/156cco2018.csv", sep=";", encoding="latin")
df_lat_lng = pd.read_csv("../data/raw/lat_lng_2018.csv")
dfs = [df, df_lat_lng[["Lat","Lng"]]]
df_final = pd.concat(dfs, axis=1)
df_final.columns = ['GRUPOSERVICO_CODIGO', 'GRUPOSERVICO_DESCRICAO', 'SERVICO_CODIGO', 'SERVICO_DESCRICAO', 
    'LOGRADOURO', 'NUMERO', 'BAIRRO', 'RPA', 'DATA_DEMANDA', 'SITUACAO', 'DATA_ULT_SITUACAO', 'LAT', 'LNG']

df_final['GRUPOSERVICO_DESCRICAO'] = df_final['GRUPOSERVICO_DESCRICAO'].astype('category')
df_final['GRUPOSERVICO_CODIGO'] = df_final['GRUPOSERVICO_CODIGO'].astype('category')
df_final['SERVICO_DESCRICAO'] = df_final['SERVICO_DESCRICAO'].astype('category')
df_final['SERVICO_CODIGO'] = df_final['SERVICO_CODIGO'].astype('category')
df_final['SITUACAO'] = df_final['SITUACAO'].astype('category')
df_final['BAIRRO'] = df_final['BAIRRO'].astype('category')
df_final['RPA'] = df_final['RPA'].astype('category')

df_final['LAT'] = df_final['LAT'].apply(lambda x: float(x) if x != 'ZERO_RESULTS' else np.NaN)
df_final['LNG'] = df_final['LNG'].apply(lambda x: float(x) if x != 'ZERO_RESULTS' else np.NaN)

df_final['DATA_ULT_SITUACAO'] = pd.to_datetime(df['DATA_ULT_SITUACAO'], format='%Y-%m-%d')
df_final['DATA_DEMANDA'] = pd.to_datetime(df['DATA_DEMANDA'], format='%Y-%m-%d')
print(df_final.dtypes)

df_final.to_hdf('../data/processed/latlng.h5', key='data', format="table")

GRUPOSERVICO_CODIGO             category
GRUPOSERVICO_DESCRICAO          category
SERVICO_CODIGO                  category
SERVICO_DESCRICAO               category
LOGRADOURO                        object
NUMERO                            object
BAIRRO                          category
RPA                             category
DATA_DEMANDA              datetime64[ns]
SITUACAO                        category
DATA_ULT_SITUACAO         datetime64[ns]
LAT                              float64
LNG                              float64
dtype: object


## Processando datasets das solicitações 156

In [27]:
dfs = []
for year in range(2012, 2019):
    path = '../data/raw/156/156cco' + str(year) + '.csv'
    df = pd.read_csv(path, sep=";", encoding="latin1")
    df['ANO'] = [year] * df.shape[0]
    dfs.append(df)
df = pd.concat(dfs)

In [28]:
mapService = {
    'CAPINAÇÃO EM VIA PAVIMENTADA': 'CAPINACAO EM VIA PAVIMENTADA',
    'CAPINAÃ\x87Ã\x83O EM VIA PAVIMENTADA': 'CAPINACAO EM VIA PAVIMENTADA',
    'CAPINAÃ\x87Ã\x83O EM VIA NÃ\x83O PAVIMENTADA': 'CAPINACAO EM VIA NAO PAVIMENTADA',
    'CAPINAÇÃO EM VIA NÃO PAVIMENTADA': 'CAPINACAO EM VIA NAO PAVIMENTADA',
    'CAPINAÇÃO EM VIA NAO PAVIMENTADA': 'CAPINACAO EM VIA NAO PAVIMENTADA',
    'DANO CAUSADO COLETA/LIXO': 'DANO CAUSADO COLETA LIXO',
    'ERRADICAÃ\x87Ã\x83O DE ARVORES': 'ERRADICACAO DE ARVORES',
    'ERRADICAÇÃO DE ARVORES': 'ERRADICACAO DE ARVORES',
    'EXUMAÃ\x87Ã\x83O': 'EXUMACAO',
    'FURTO EM TÃ\x9aMULOS': 'FURTO EM TUMULOS',
    'IMPLANTAÇÃO LAMPADA LED': 'IMPLANTACAO LAMPADA LED',
    'INSTALAÃ\x87Ã\x83O DE CONTENEDORES DE LIXO': 'INSTALACAO DE CONTENEDORES DE LIXO',
    'INSTALAÇÃO DE CONTENEDORES DE LIXO': 'INSTALACAO DE CONTENEDORES DE LIXO',
    'INSTALAÇÃO PROVISÓRIA DE PONTO DE AC': 'INSTALACAO PROVISORIA DE PONTO DE AC',
    'INSTALAÃ\x87Ã\x83O PROVISÃ\x93RIA DE PONTO DE AC': 'INSTALACAO PROVISORIA DE PONTO DE AC',
    'INSTALAÃ\x87Ã\x83O PROVISÃ\x93RIA DE QUADRO': 'INSTALACAO PROVISORIA DE QUADRO',
    'INSTALAÇÃO PROVISÓRIA DE QUADRO': 'INSTALACAO PROVISORIA DE QUADRO',
    'LIMP.DE GAL,VALETAS,SARJETAS': 'LIMP. DE GALERIAS,VALETAS,SARJETAS',
    'LIMPEZA DE  CANALETAS': 'LIMPEZA DE CANALETAS',
    'MANUTENÃ\x87Ã\x83O EM LÃ\x82MPADA APAGADA': 'MANUTENCAO EM LAMPADA APAGADA',
    'MANUTENÇÃO EM LÂMPADA APAGADA': 'MANUTENCAO EM LAMPADA APAGADA', 
    'MUDANÃ\x87A DE PONTO DE CONFINAMENTO DE LIXO': 'MUDANCA DE PONTO DE CONFINAMENTO DE LIXO',
    'MUNDANÃ\x87A DE PONTO DE CONFINAMENTO DE LIXO': 'MUDANCA DE PONTO DE CONFINAMENTO DE LIXO',
    'MUDANÇA DE PONTO DE CONFINAMENTO DE LIXO': 'MUDANCA DE PONTO DE CONFINAMENTO DE LIXO',
    'OPERAÃ\x87Ã\x83O TAPA-BURACO EM ASFALTO': 'OPER. TAPA-BURACO EM ASFALTO',
    'OPERAÇÃO TAPA-BURACO EM ASFALTO': 'OPER. TAPA-BURACO EM ASFALTO',
    'RECUPERAÃ\x87Ã\x83O DE PAVIMENTO PARALELEPIPEDO': 'RECUPERACAO DE PAVIMENTO PARALELEPIPEDO',
    'REMOÃ\x87Ã\x83O DE ANIMAIS MORTOS DE GRANDE PORTE': 'REMOCAO DE ANIMAIS MORTOS DE GRANDE PORTE',
    'REMOÇÃO DE ANIMAIS MORTOS DE GRANDE PORTE': 'REMOCAO DE ANIMAIS MORTOS DE GRANDE PORTE',
    'REMOCAO ANIMAIS MORTOS PEQUENOS': 'REMOCAO DE ANIMAIS MORTOS PEQUENOS',
    'REMOÃ\x87Ã\x83O DE ANIMAIS MORTOS PEQUENOS': 'REMOCAO DE ANIMAIS MORTOS PEQUENOS',
    'REMOÇÃO DE ANIMAIS MORTOS PEQUENOS': 'REMOCAO DE ANIMAIS MORTOS PEQUENOS',
    'REMOÃ\x87Ã\x83O DE ENTULHOS': 'REMOCAO DE ENTULHOS',
    'REMOÇÃO DE ENTULHOS': 'REMOCAO DE ENTULHOS',
    'REMOÃ\x87Ã\x83O DE PONTO CRITICO': 'REMOCAO DE PONTO CRITICO',
    'REMOÇÃO DE PONTO CRITICO': 'REMOCAO DE PONTO CRITICO',
    'REMOÃ\x87Ã\x83O DE UTILITÃ\x81RIOS DOMESTICOS GELADEIRA,SOFA,ETC': 'REMOCAO DE UTILITARIOS DOMESTICOS GELADEIRA,SOFA,ETC',
    'REMOÇÃO DE UTILITÁRIOS DOMESTICOS GELADEIRA,SOFA,ETC': 'REMOCAO DE UTILITARIOS DOMESTICOS GELADEIRA,SOFA,ETC',
    'REPOSIÃ\x87Ã\x83O TAMPA DE POÃ\x87O, GALERIAS E PRE-MOLDADOS': 'REPOS.TAMPA POCO, GALER E PRE-MOLDADOS',
    'REPOSIÇÃO TAMPA DE POÇO, GALERIAS E PRE-MOLDADOS': 'REPOS.TAMPA POCO, GALER E PRE-MOLDADOS',
    'PROJETO DE ILUMINAÃ\x87Ã\x83O': 'PROJETO DE ILUMINACAO',
    'RECUPERAÇÃO DE PAVIMENTO PARALELEPIPEDO': 'RECUPERACAO DE PAVIMENTO PARALELEPIPEDO',
}

df['SERVICO_DESCRICAO'] = df['SERVICO_DESCRICAO'].apply(lambda x: mapService[x] if x in mapService else x)

In [29]:
mapGroup = {
    'ARBORIZAÃ\x87Ã\x83O': 'ARBORIZACAO',
    'ARBORIZAÇÃO': 'ARBORIZACAO',
    'CALÃ\x87ADAS ARVORES': 'CALCADAS ARVORES',
    'CALÇADAS ARVORES': 'CALCADAS ARVORES',
    'FISCALIZAÃ\x87Ã\x83O URBANA': 'FISCALIZACAO URBANA',
    'FISCALIZAÇÃO URBANA': 'FISCALIZACAO URBANA',
    'ILUMINAÃ\x87Ã\x83O PROVISÃ\x93RIA': 'ILUMINACAO PROVISORIA',
    'ILUMINAÃ\x87Ã\x83O PÃ\x9aBLICA': 'ILUMINACAO PUBLICA',
    'ILUMINAÃ\x87Ã\x83O RELUZ': 'ILUMINACAO RELUZ',
    'ILUMINAÇÃO PROVISÓRIA': 'ILUMINACAO PROVISORIA',
    'ILUMINAÇÃO PÚBLICA': 'ILUMINACAO PUBLICA',
    'ILUMINAÇÃO RELUZ': 'ILUMINACAO RELUZ',
    'LUMINÃ\x81RIAS': 'LUMINARIAS',
    'LUMINÁRIAS': 'LUMINARIAS',
    'PAVIMENTAÃ\x87Ã\x83O': 'PAVIMENTACAO',
    'PAVIMENTAÇÃO': 'PAVIMENTACAO',
    'PRAÃ\x87AS': 'PRACAS',
    'PRAÇAS': 'PRACAS',
    'VÃ\x83O DE REDE': 'VAO DE REDE',
    'VÃO DE REDE': 'VAO DE REDE'}

df['GRUPOSERVICO_DESCRICAO'] = df['GRUPOSERVICO_DESCRICAO'].apply(lambda x: mapGroup[x] if x in mapGroup else x)

In [33]:
mapSituacao = {
    'EXECUÃ\x87Ã\x83O': 'EXECUCAO',
    'EXECUÇÃO': 'EXECUCAO',
    'FISCALIZAÃ\x87Ã\x83O': 'FISCALIZACAO',
    'FISCALIZAÇÃO': 'FISCALIZACAO',
    'PREPARAÃ\x87Ã\x83O': 'PREPARACAO',
    'PREPARAÇÃO': 'PREPARACAO',
}

df['SITUACAO'] = df['SITUACAO'].apply(lambda x: mapSituacao[x] if x in mapSituacao else x)

In [36]:
len(set(df['BAIRRO']))

94

In [51]:
df['GRUPOSERVICO_DESCRICAO'] = df['GRUPOSERVICO_DESCRICAO'].astype('category')
df['GRUPOSERVICO_CODIGO'] = df['GRUPOSERVICO_CODIGO'].astype('category')
df['SERVICO_DESCRICAO'] = df['SERVICO_DESCRICAO'].astype('category')
df['SERVICO_CODIGO'] = df['SERVICO_CODIGO'].astype('category')
df['SITUACAO'] = df['SITUACAO'].astype('category')
df['BAIRRO'] = df['BAIRRO'].astype('category')
df['RPA'] = df['RPA'].astype('category')

df['DATA_ULT_SITUACAO'] = pd.to_datetime(df['DATA_ULT_SITUACAO'], format='%Y-%m-%d')
df['DATA_DEMANDA'] = pd.to_datetime(df['DATA_DEMANDA'], format='%Y-%m-%d')
print(df.dtypes)

df.to_hdf('../data/processed/solicitacoes156.h5', key='data', format="table")

GRUPOSERVICO_CODIGO             category
GRUPOSERVICO_DESCRICAO          category
SERVICO_CODIGO                  category
SERVICO_DESCRICAO               category
LOGRADOURO                        object
NUMERO                            object
BAIRRO                          category
RPA                             category
DATA_DEMANDA              datetime64[ns]
SITUACAO                        category
DATA_ULT_SITUACAO         datetime64[ns]
ANO                                int64
dtype: object
